In [60]:
import gurobipy as gp
from gurobipy import GRB

# Factory Planning I:


In [65]:

products = ["Prod1", "Prod2", "Prod3", "Prod4", "Prod5", "Prod6", "Prod7"]
machines = ["grinder", "vertDrill", "horiDrill", "borer", "planer"]
time_periods = ["January", "February", "March", "April", "May", "June"]

profit_contribution = {"Prod1":10, "Prod2":6, "Prod3":8, "Prod4":4, "Prod5":11, "Prod6":9, "Prod7":3}

time_table = {
    "grinder": {    "Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3,
                    "Prod6": 0.2, "Prod7": 0.5 },
    "vertDrill": {  "Prod1": 0.1, "Prod2": 0.2, "Prod4": 0.3,
                    "Prod6": 0.6 },
    "horiDrill": {  "Prod1": 0.2, "Prod3": 0.8, "Prod7": 0.6 },
    "borer": {      "Prod1": 0.05,"Prod2": 0.03,"Prod4": 0.07,
                    "Prod5": 0.1, "Prod7": 0.08 },
    "planer": {     "Prod3": 0.01,"Prod5": 0.05,"Prod7": 0.05 }
}


# number of machines down
down = {("January","grinder"): 1, ("February", "horiDrill"): 2, ("March", "borer"): 1,
        ("April", "vertDrill"): 1, ("May", "grinder"): 1, ("May", "vertDrill"): 1,
        ("June", "planer"): 1, ("June", "horiDrill"): 1}

# number of each machine available
qMachine = {"grinder":4, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} 

# market limitation of sells
upper = {
    ("January", "Prod1") : 500,
    ("January", "Prod2") : 1000,
    ("January", "Prod3") : 300,
    ("January", "Prod4") : 300,
    ("January", "Prod5") : 800,
    ("January", "Prod6") : 200,
    ("January", "Prod7") : 100,
    ("February", "Prod1") : 600,
    ("February", "Prod2") : 500,
    ("February", "Prod3") : 200,
    ("February", "Prod4") : 0,
    ("February", "Prod5") : 400,
    ("February", "Prod6") : 300,
    ("February", "Prod7") : 150,
    ("March", "Prod1") : 300,
    ("March", "Prod2") : 600,
    ("March", "Prod3") : 0,
    ("March", "Prod4") : 0,
    ("March", "Prod5") : 500,
    ("March", "Prod6") : 400,
    ("March", "Prod7") : 100,
    ("April", "Prod1") : 200,
    ("April", "Prod2") : 300,
    ("April", "Prod3") : 400,
    ("April", "Prod4") : 500,
    ("April", "Prod5") : 200,
    ("April", "Prod6") : 0,
    ("April", "Prod7") : 100,
    ("May", "Prod1") : 0,
    ("May", "Prod2") : 100,
    ("May", "Prod3") : 500,
    ("May", "Prod4") : 100,
    ("May", "Prod5") : 1000,
    ("May", "Prod6") : 300,
    ("May", "Prod7") : 0,
    ("June", "Prod1") : 500,
    ("June", "Prod2") : 500,
    ("June", "Prod3") : 100,
    ("June", "Prod4") : 300,
    ("June", "Prod5") : 1100,
    ("June", "Prod6") : 500,
    ("June", "Prod7") : 60,
}


storeCost = 0.5
storeCapacity = 100
endStock = 50
hoursPerMonth = 2*8*24 #each month has 24 working days.


384

##  Single Period Formulation:

We're dealing with a complex problem, lets try to solve a smaller sub-problem first. 
Lets just try to figure out the best use of machine hours for just january. We'll then mix in temporal elements of this question. 




The Objective Function is the calculation of profit: the number of products $p$ we choose to sell at their price, minus the cost of storing the amount of product $p$ we choose to store.  

$$max \sum_{p \in Products} Profit_p \cdot Sell_p - \sum_{p \in Products}costStorage \cdot Store_p$$

We have a lot of decision variables:

1. $Sell_p$ represents the amount we choose to sell of product $p$
2. $Store_p$ represents the amount we choose to store of product $p$
3. $Make_p$ represents the amount of product $p$ we choose to make

We're going to let $Profit_p$ represent the contribution to profit product $p$ yields, and $machine_{i,p}$ represents the amount of hours machine $i$ needs to product product $p$. 

Our first constraint regards the first three decision variables. 
$$Store_p = Make_p - Sell_p$$

Second, we create a group of constraints for each machine. Given that we have five types of machines, each with a coefficient of how much it takes to make something per hour, the most each machine can work is the maximum hours-- and if all of those machines are working simotenously, then the most all of those machines can work is the number of machines * maximum hours. 
This follows with the following constraint, for each type of machine $m$:
$$ \sum_{ p \in \{products\}} p \cdot m_{coeff} \leq Num_m \cdot maxhours $$

where necessaryMachines(p) is a function that returns the necessary machines to create product $p$.

Third, for each $p$, we initialize the marketing limitations:
$$ Sell_p \leq MarketLimit_p$$

Fourth, for each $p$, we add the storage limitations:
$$ Store_p \leq 100$$

Finally, we need to ensure that the total hours across all machines is less than or equal to the maximum hours the factory is open to work:
$$\sum_{i \in all machines} hours_i  \leq MaxHours$$



In [62]:
#helper function
def calculate_availible_machines(input_month):
    q_machine = {"grinder":4, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} 

    for month,machine in down.keys():
        if month == input_month:
            for qmach in q_machine.keys():
                if qmach == machine:
                    q_machine[qmach] -= down[(month,machine)]
    return q_machine


[(<gurobi.Var grinder_i_0 (value 0.0)>, 0.5),
 (<gurobi.Var grinder_i_1 (value 1.0)>, 0.5),
 (<gurobi.Var grinder_i_2 (value 2.0)>, 0.5),
 (<gurobi.Var grinder_i_3 (value 378.0)>, 0.5),
 (<gurobi.Var vertDrill_i_0 (value 0.0)>, 0.1),
 (<gurobi.Var horiDrill_i_0 (value 0.0)>, 0.2),
 (<gurobi.Var horiDrill_i_1 (value 1.0)>, 0.2),
 (<gurobi.Var horiDrill_i_2 (value 2.0)>, 0.2),
 (<gurobi.Var borer_i_0 (value 0.0)>, 0.05)]

In [86]:
current_month = "April"

try:
    m = gp.Model()
    sell = m.addVars(products,vtype=GRB.CONTINUOUS, name='Sell')
    store = m.addVars(products,vtype=GRB.CONTINUOUS, name='Store')
    make = m.addVars(products,vtype=GRB.CONTINUOUS, name='Make')
    
    total_machines_availible = calculate_availible_machines(current_month)

    
    
    m.update()
    #profit calc
    obj = sell.prod(profit_contribution) - storeCost *store.sum() 
    m.setObjective(obj,GRB.MAXIMIZE)
    
    storeConstr = m.addConstrs(store[prod] == make[prod] - sell[prod] for prod in products)
    

    m.addConstrs(sell[p] <= upper[(current_month,p)] for p in products)
    
    m.addConstrs(store[p] <= storeCapacity for p in products)
    
    for machine in time_table.keys():
        machine_multiples = []
        for prod in time_table[machine].keys():
            if prod in make.keys():
                machine_multiples.append( (time_table[machine][prod], make[prod]))
        m.addConstr( gp.quicksum([x[0]*x[1] for x in machine_multiples]) <= qMachine[machine] *hoursPerMonth)
    
    m.update()
    m.optimize()
    
    
    
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))






Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 26 rows, 21 columns and 55 nonzeros
Model fingerprint: 0x620e1684
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e-01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 26 rows and 21 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.150000000e+04


In [87]:
for i in range(len(m.getVars())):
    print(f'{m.getVars()[i].VarName} : {m.X[i]}')

Sell[Prod1] : 200.0
Sell[Prod2] : 300.0
Sell[Prod3] : 400.0
Sell[Prod4] : 500.0
Sell[Prod5] : 200.0
Sell[Prod6] : 0.0
Sell[Prod7] : 100.0
Store[Prod1] : 0.0
Store[Prod2] : 0.0
Store[Prod3] : 0.0
Store[Prod4] : 0.0
Store[Prod5] : 0.0
Store[Prod6] : 0.0
Store[Prod7] : 0.0
Make[Prod1] : 200.0
Make[Prod2] : 300.0
Make[Prod3] : 400.0
Make[Prod4] : 500.0
Make[Prod5] : 200.0
Make[Prod6] : 0.0
Make[Prod7] : 100.0


It seems that for the single period problem, we make only as much as we need to sell. This cuts down costs. But what if the marketing limitations for next month exceed our capacity to make things next month? How could we make an algorithm that can optimially schedule the production of our products with respect to time?

## Multi-period formulation:
Adding in time increases the complexity of this problem, but still keeps it within polynomial time, as it is still a linear program. 

Our objective function is to maximize profit in the given time period (six months), which leads to its mathematical formulation:
$$\sum_t \sum_{p \in \{products\}} (Sell^{(t)}_p - 0.5 Store^{(t)}_p) $$
for all t time periods.

Our decision variables are similiar-- we decide how much to make, store, and sell, but within the context of a specific month.
1. $make^{(t)}_p$ represents how much product $p$ is made at time $t$
2. $store^{(t)}_p$ represents how much product $p$ is stored after selling and making $p$ at time $t$
3. $sell^{(t)}_p$ represents how much product $p$ is sold at time $t$

We start with zero of every product in storage. Therefore, for each $t$, we are able to relate each month through linking constraints:
$$ store^{(t)}_p = make^{(t)}_p + store^{(t-1)}_p - sell^{(t)}_p$$
In english, how much we choose to store at time $t$ is the sum of how much of $p$ we had in storage and how much of product $p$ we chose to make minus the amount of $p$ we chose to sell.

As the products themselves don't change, the coefficients of each of the products dont change either. However, the production constraints of each of the machines changes as some machines are down for maintenance each month. Moreover, how much we choose to make each month should not only depend on how much we can sell, but also on what machines are down each month. Therefore, this leads to the following constraint:



['Prod1', 'Prod2', 'Prod3', 'Prod4', 'Prod5', 'Prod6', 'Prod7']